In [ ]:
class CFG:
    exp = '006'
    debug=False
    seed = 42

# ====================================================
# Colab settings
# ====================================================
class ColabConfig:
    dataset_name = 'PPPM-stacking'
    dataset_version = 'exp' + CFG.exp
    dataset_new = True  # 新しいデータセットか
    dataset_dir = None  # Kaggle Dataset にアップロードするディレクトリ
    dataset_note = '""'  # 前の版からの変更点
    in_colab = False  # colab上にデータダウンロード

In [ ]:
!nvcc --version
!python -c 'import torch; print(torch.__version__) '
!python --version
print('')
!nvidia-smi
print('')
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
1.11.0+cu113
Python 3.7.13

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.


Your runtime has 13.6 gigabytes of available RAM



In [ ]:
%%time
import sys
COLAB = "google.colab" in sys.modules

if COLAB:
    import os
    print('This environment is Google Colab')

    # mount drive
    from google.colab import drive
    if not os.path.isdir('/content/drive'):
        drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/kaggle/PPPM/exps')

    # kaggle api token and update kaggle api
    from google.colab import files
    if not os.path.isfile('~/.kaggle/kaggle.json'):
        # files.upload()
        !mkdir -p ~/.kaggle
        !cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install --upgrade --force-reinstall --no-deps -q kaggle

    if ColabConfig.in_colab:
        # make directory in colab
        !mkdir -p /content/input
        !mkdir -p /content/working

        # download dataset in colab
        import zipfile, glob
        os.chdir('/content/input')
        # !kaggle competitions download -qc birdclef-2022 -p birdclef-2022
        
        # !mkdir birdclef-2022
        # !cp /content/drive/MyDrive/kaggle/BirdCLEF2022/input/birdclef-2022/* ./birdclef-2022/

        for p in glob.glob('**/*.zip', recursive=True):
            print(p)
            d, f = os.path.split(p)
            # if f in ['']:
            #     continue
            with zipfile.ZipFile(p, 'r') as zipf:
                print('unzip: ', zipf)
                zipf.extractall(d)
                print('remove: ', f)
                os.remove(p)
        
        os.chdir('/content/working')

This environment is Google Colab
Mounted at /content/drive
     |████████████████████████████████| 58 kB 4.1 MB/s 
CPU times: user 1.67 s, sys: 359 ms, total: 2.03 s
Wall time: 26.1 s


In [ ]:
import os
import random
import pickle
from pathlib import Path
from psutil import cpu_count
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', None)
from tqdm.auto import tqdm
tqdm.pandas()
os.system('python -m pip install -U lightgbm')
import lightgbm as lgb
print(f"lightgbm.__version__: {lgb.__version__}")
os.system('python -m pip install optuna')
import optuna.integration.lightgbm as lgb_optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import torch
print(f"torch.__version__: {torch.__version__}")

os.system('python -m pip install sentencepiece')
os.system('pip uninstall -y transformers')
os.system('pip uninstall -y tokenizers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels transformers')
os.system('python -m pip install --no-index --find-links=../input/pppm-pip-wheels tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

lightgbm.__version__: 3.3.2
torch.__version__: 1.11.0+cu113
tokenizers.__version__: 0.12.1
transformers.__version__: 4.18.0
env: TOKENIZERS_PARALLELISM=true


In [ ]:
input_dir = Path('../input/us-patent-phrase-to-phrase-matching')
oof_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/{CFG.exp}')
output_dir = Path(f'/content/drive/MyDrive/kaggle/PPPM/output/stacking{CFG.exp}')
output_dir.mkdir(exist_ok=True)

ColabConfig.dataset_dir = str(output_dir)

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


def get_logger(filename=output_dir / 'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

# LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [24]:
oof_df = pd.read_pickle(oof_dir / 'oof_df.pkl')
print(get_score(oof_df['score'], oof_df['pred']))
oof_df.head()

0.5023895431810046


,id,anchor,target,context,score,context_text,fold,pred
0,37d61fd2272659b1,abatement.,abatement of pollution.,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.049280
1,7b9652b17b68b7a4,abatement.,act of abating.,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.262063
2,36d72442aefd8232,abatement.,active catalyst.,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.406801
3,5296b0c19e1ce60e,abatement.,eliminating process.,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.362485
4,54c1e3b9184cb5b6,abatement.,forest region.,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.002926


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(oof_dir / 'tokenizer')

In [ ]:
train = oof_df.copy()

# 特徴量エンジニアリング
def tok_len(sentence):
    return len(tokenizer.tokenize(sentence))

def feature_engineering(df):
    df['section'] = df['context'].map(lambda s: s[0])

    df['anchor_tok_len'] = df['anchor'].map(tok_len)
    df['target_tok_len'] = df['target'].map(tok_len)
    df['context_tok_len'] = df['context_text'].map(tok_len)
    df['input_len'] = df['anchor_tok_len'] + df['target_tok_len'] + df['context_tok_len'] + 4

    df['len_anc_tgt_diff'] = df['anchor_tok_len'] - df['target_tok_len']
    df['len_anc_tgt_div'] = df['anchor_tok_len'] / df['target_tok_len']

    df['len_anc_cnt_diff'] = df['anchor_tok_len'] - df['context_tok_len']
    df['len_anc_cnt_div'] = df['anchor_tok_len'] / df['context_tok_len']

    df['len_tgt_cnt_diff'] = df['target_tok_len'] - df['context_tok_len']
    df['len_tgt_cnt_div'] = df['target_tok_len'] / df['context_tok_len']

    return df


train = feature_engineering(train)
train.head()

,id,anchor,target,context,score,context_text,fold,pred,section,anchor_tok_len,target_tok_len,context_tok_len,input_len,len_anc_tgt_diff,len_anc_tgt_div,len_anc_cnt_diff,len_anc_cnt_div,len_tgt_cnt_diff,len_tgt_cnt_div
0,37d61fd2272659b1,abatement.,abatement of pollution.,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.049280,A,2,4,20,30,-2,0.500000,-18,0.1,-16,0.20
1,7b9652b17b68b7a4,abatement.,act of abating.,A47,0.75,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.262063,A,2,6,20,32,-4,0.333333,-18,0.1,-14,0.30
2,36d72442aefd8232,abatement.,active catalyst.,A47,0.25,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.406801,A,2,3,20,29,-1,0.666667,-18,0.1,-17,0.15
3,5296b0c19e1ce60e,abatement.,eliminating process.,A47,0.50,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.362485,A,2,3,20,29,-1,0.666667,-18,0.1,-17,0.15
4,54c1e3b9184cb5b6,abatement.,forest region.,A47,0.00,HUMAN NECESSITIES. FURNITURE. DOMESTIC ARTICLES OR APPLIANCES. COFFEE MILLS. SPICE MILLS. SUCTION CLEANERS IN GENERAL,0,0.002926,A,2,3,20,29,-1,0.666667,-18,0.1,-17,0.15


In [ ]:
# カテゴリ変数の変換
cat_cols = ['section']
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[f'{c}_encoded'] = le.transform(train[c])
    with open(output_dir / f'{c}_encoder.pkl', 'wb') as f:
        pickle.dump(le, f)

In [ ]:
print(train.columns)

Index(['id', 'anchor', 'target', 'context', 'score', 'context_text', 'fold', 'pred', 'section', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded'], dtype='object')


In [ ]:
excluded_columns = ['id', 'anchor', 'target', 'context', 'score', 'context_text', 'text', 'text_rev', 'fold', 'section']
cols = [col for col in train.columns if col not in excluded_columns]
with open(output_dir / 'cols.pkl', 'wb') as f:
    pickle.dump(cols, f)
print(cols)

['pred', 'anchor_tok_len', 'target_tok_len', 'context_tok_len', 'input_len', 'len_anc_tgt_diff', 'len_anc_tgt_div', 'len_anc_cnt_diff', 'len_anc_cnt_div', 'len_tgt_cnt_diff', 'len_tgt_cnt_div', 'section_encoded']


In [ ]:
params = {
    'objective': 'regression',
    'boosting': 'gbdt',
    'num_iterations': 1000,
    'learning_rate': 0.01,
    # 'num_leaves': 20,  # 31
    'num_threads': cpu_count(logical=False),
    'seed': CFG.seed,

    # 'max_depth': 3,  # -1
    # 'min_data_in_leaf': 100,  # 20
    'verbosity': -1,  # 1

    'metric': 'l2'
}

In [ ]:
def tuning_params(df, params):
    anchors = df['anchor'].unique()
    rng = np.random.default_rng(seed=71)
    rng.shuffle(anchors)
    val_prop = 0.25  # バリデーションデータの割合。4foldから0.25
    val_size = int(len(anchors)*val_prop)
    val_anchors = anchors[:val_size]
    is_val = np.isin(df['anchor'], val_anchors)
    idx = np.arange(len(df))
    val_idx = idx[is_val]
    tr_idx = idx[~is_val]
    print(f'train: {len(tr_idx)}, val: {len(val_idx)}')

    tr_df = df.iloc[tr_idx]
    val_df = df.iloc[val_idx]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)
    
    model = lgb_optuna.train(
        params,
        tr_ds, 
        valid_sets=[tr_ds, val_ds],
        callbacks=[lgb.early_stopping(stopping_rounds=100),
                    lgb.log_evaluation(period=100)],
        verbosity=-1,
    )

    best_params = model.params

    return best_params

In [ ]:
best_params = tuning_params(train, params)
best_params

[I 2022-06-07 14:21:02,917] A new study created in memory with name: no-name-230e80b4-da12-4191-851e-20506d54bf64


train: 6840, val: 2539


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l2: 0.0653918	valid_1's l2: 0.0673789
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l2: 0.0652829	valid_1's l2: 0.0672527
[3]	valid_0's l2: 0.0648589	valid_1's l2: 0.0668839
[4]	valid_0's l2: 0.0644439	valid_1's l2: 0.0665285
[5]	valid_0's l2: 0.0640381	valid_1's l2: 0.0661778
[6]	valid_0's l2: 0.063639	valid_1's l2: 0.0658333
[7]	valid_0's l2: 0.0632473	valid_1's l2: 0.0655012
[8]	valid_0's l2: 0.063149	valid_1's l2: 0.0653929
[9]	valid_0's l2: 0.062768	valid_1's l2: 0.0650586
[10]	valid_0's l2: 0.0623929	valid_1's l2: 0.0647381
[11]	valid_0's l2: 0.0620253	valid_1's l2: 0.0644274
[12]	valid_0's l2: 0.0616657	valid_1's l2: 0.0641175
[13]	valid_0's l2: 0.0613128	valid_1's l2: 0.0638177
[14]	valid_0's l2: 0.0609652	valid_1's l2: 0.0635254
[15]	valid_0's l2: 0.060626	valid_1's l2: 0.0632314
[16]	valid_0's l2: 0.0602938	valid_1's l2: 0.0629465
[17]	valid_0's l2: 0.0599671	valid_1's l2: 0.0626705
[18]	valid_0's l2: 0.0596479	valid_1's l2: 0.06239

feature_fraction, val_score: 0.050071:  14%|#4        | 1/7 [00:00<00:04,  1.24it/s]

...

[323]	valid_0's l2: 0.0437713	valid_1's l2: 0.0492585
[324]	valid_0's l2: 0.0437607	valid_1's l2: 0.0492612
[325]	valid_0's l2: 0.0437555	valid_1's l2: 0.0492615
[326]	valid_0's l2: 0.0437481	valid_1's l2: 0.0492618
[327]	valid_0's l2: 0.0437388	valid_1's l2: 0.049264
[328]	valid_0's l2: 0.0437315	valid_1's l2: 0.0492657
[329]	valid_0's l2: 0.0437228	valid_1's l2: 0.0492677
[330]	valid_0's l2: 0.043716	valid_1's l2: 0.0492691
[331]	valid_0's l2: 0.0437079	valid_1's l2: 0.0492744
[332]	valid_0's l2: 0.0437021	valid_1's l2: 0.0492748
[333]	valid_0's l2: 0.043697	valid_1's l2: 0.0492741
[334]	valid_0's l2: 0.0436905	valid_1's l2: 0.0492746
[335]	valid_0's l2: 0.0436831	valid_1's l2: 0.0492793
[336]	valid_0's l2: 0.0436756	valid_1's l2: 0.0492833
[337]	valid_0's l2: 0.0436698	valid_1's l2: 0.0492823
[338]	valid_0's l2: 0.0436609	valid_1's l2: 0.0492865
[339]	valid_0's l2: 0.0436562	valid_1's l2: 0.0492857
[340]	valid_0's l2: 0.0436478	valid_1's l2: 0.0492897
[341]	valid_0's l2: 0.0436405	v

min_data_in_leaf, val_score: 0.049235:  80%|########  | 4/5 [00:04<00:00,  1.13it/s]

[321]	valid_0's l2: 0.0439181	valid_1's l2: 0.0492586
[322]	valid_0's l2: 0.0439089	valid_1's l2: 0.0492567
[323]	valid_0's l2: 0.0438998	valid_1's l2: 0.049255
[324]	valid_0's l2: 0.0438906	valid_1's l2: 0.0492571
[325]	valid_0's l2: 0.0438852	valid_1's l2: 0.0492574
[326]	valid_0's l2: 0.0438781	valid_1's l2: 0.049257
[327]	valid_0's l2: 0.0438711	valid_1's l2: 0.0492585
[328]	valid_0's l2: 0.0438634	valid_1's l2: 0.0492607
[329]	valid_0's l2: 0.0438566	valid_1's l2: 0.0492639
[330]	valid_0's l2: 0.0438504	valid_1's l2: 0.0492653
[331]	valid_0's l2: 0.0438431	valid_1's l2: 0.0492695
[332]	valid_0's l2: 0.0438368	valid_1's l2: 0.0492698
[333]	valid_0's l2: 0.0438321	valid_1's l2: 0.0492696
[334]	valid_0's l2: 0.0438262	valid_1's l2: 0.0492703
[335]	valid_0's l2: 0.0438191	valid_1's l2: 0.0492744
[336]	valid_0's l2: 0.0438125	valid_1's l2: 0.049279
[337]	valid_0's l2: 0.0438067	valid_1's l2: 0.0492775
[338]	valid_0's l2: 0.0437982	valid_1's l2: 0.0492843
[339]	valid_0's l2: 0.0437934	v

min_data_in_leaf, val_score: 0.049235: 100%|##########| 5/5 [00:04<00:00,  1.11it/s]


{'bagging_fraction': 0.7708200400954313,
 'bagging_freq': 5,
 'boosting': 'gbdt',
 'categorical_column': [11],
 'early_stopping_round': None,
 'feature_fraction': 0.7200000000000001,
 'feature_pre_filter': False,
 'lambda_l1': 0.2241095526472094,
 'lambda_l2': 9.505247295587479,
 'learning_rate': 0.01,
 'metric': 'l2',
 'min_child_samples': 20,
 'num_iterations': 1000,
 'num_leaves': 11,
 'num_threads': 1,
 'objective': 'regression',
 'seed': 42,
 'verbosity': -1}

In [ ]:
def prepare_datasets(df, fold):
    tr_df = df[df['fold']!=fold]
    val_df = df[df['fold']==fold]

    cat_cols_ = [f'{c}_encoded' for c in cat_cols]
    tr_ds = lgb.Dataset(tr_df[cols], tr_df['score'], categorical_feature=cat_cols_)
    val_ds = lgb.Dataset(val_df[cols], val_df['score'], categorical_feature=cat_cols_, reference=tr_ds)

    return tr_ds, val_ds


def train_fn(df, params):
    preds = []

    for fold in df['fold'].unique():
        tr_ds, val_ds = prepare_datasets(df, fold)
        
        model = lgb.train(
            params, 
            tr_ds, 
            valid_sets=[tr_ds, val_ds],
            callbacks=[lgb.early_stopping(stopping_rounds=100),
                       lgb.log_evaluation(period=100)]
        )

        pred = df[df['fold']==fold].copy()
        pred['pred_lgb'] = model.predict(pred[cols], num_iteration=model.best_iteration)
        preds.append(pred)

        model.save_model(output_dir / f'lgb_fold{fold}.txt', num_iteration=model.best_iteration)

    return pd.concat(preds, axis=0, ignore_index=False)

In [ ]:
pred = train_fn(train, best_params)

ValueError: ignored

In [ ]:
importances = []
for fold in train['fold'].unique():
    importances.append(
        lgb.Booster(model_file=output_dir / f'lgb_fold{fold}.txt').feature_importance(importance_type='gain'))
importance_df = pd.DataFrame({
    'feature': cols,
    'importance': np.mean(importances, axis=0)
    })
importance_df.sort_values('importance', ascending=False)

In [ ]:
# optunaのbest params
print(get_score(pred['score'], pred['pred']))
print(get_score(pred['score'], pred['pred_lgb']))
pred.head()

In [ ]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

ID = 'hanejiyuto'
DATASET_ID = ColabConfig.dataset_name + '-' + ColabConfig.dataset_version
UPLOAD_DIR = ColabConfig.dataset_dir
VERSION_NOTES = ColabConfig.dataset_note

def dataset_create_new():
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_new(folder=UPLOAD_DIR, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets create -t -p $UPLOAD_DIR -r tar

def dataset_create_version():  # バージョンアップデート
    dataset_metadata = {}
    dataset_metadata['id'] = f'{ID}/{DATASET_ID}'
    dataset_metadata['licenses'] = [{'name': 'CC0-1.0'}]
    dataset_metadata['title'] = DATASET_ID
    if not os.path.exists('dataset-metadata.json'):
        with open(os.path.join(UPLOAD_DIR, 'dataset-metadata.json'), 'w') as f:
            json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    # api.dataset_create_version(folder=UPLOAD_DIR, version_notes=VERSION_NOTES, convert_to_csv=False, dir_mode='tar')
    !kaggle datasets version -t -p $UPLOAD_DIR -r tar -m $VERSION_NOTES

if CFG.debug:
    pass
elif ColabConfig.dataset_new:
    dataset_create_new()
else:
    dataset_create_version()